In [1]:
from opcua import Server

server = Server()
server.set_endpoint("opc.tcp://localhost:4840/freeopcua/server/")    # 设定服务器URI
uri = 'http://examples.freeopcua.github.io'
idx = server.register_namespace(uri)    # 注册地址空间

server.import_xml("log_test.xml")    # 导入自定义的节点类型

#读取xml配置实例化自定义对象
#my_sensor_type = server.get_root_node().get_child([
#    "0:Types", "0:ObjectTypes", "0:BaseObjectType", "0:SysType"]).nodeid
#my_sensor = server.nodes.objects.add_object(idx, "Sys", my_sensor_type)

server.start()


Endpoints other than open requested but private key and certificate are not set.
Listening on localhost:4840


In [2]:
#读取测试文件
file=open('syslog_test.txt')    
dataMat=[]  
for line in file.readlines():    
    curLine=line.strip('\n')     
    dataMat.append(curLine) 

In [3]:
#实例化对象
myobj = server.nodes.objects.add_object(idx, "sys")#直接实例化对象以及添加变量
myvar = myobj.add_variable(idx, "Single log",dataMat[0] )

Unknown message received FourByteNodeId(i=12208)


In [4]:
#实例化对象（先不用）
def InsData(str):
    myobj = server.nodes.objects.add_object(idx, "sys")#直接实例化对象以及添加变量
    myvar = myobj.add_variable(idx, "Single log", str)


In [6]:
#简单更新一下
def upData(str):
    myvar.set_value(str)

In [7]:
#方法1：定时器
def fun_timer():
    global i
    i=i+1
    #InsData(dataMat[i])
    upData(dataMat[i])
    print(dataMat[i])
    global timer
    timer = threading.Timer(3.5, fun_timer)#每十秒刷新一次数据
    timer.start()

In [6]:
#方法2：实现定时更新数据功能（循环内暂停的方法）
from threading import Thread
import logging
from datetime import datetime
import time
class VarUpdater(Thread):
    def __init__(self,var):
        Thread.__init__(self)
        self._stopev = False
        self.var = var

    def stop(self):
        self._stopev = True

    def run(self):
        global i
        while not self._stopev:
            v = dataMat[i]
            self.var.set_value(v) #更新值
            i=i+1
            time.sleep(3)

In [8]:
#方法1：启动定时器
import threading
timer = threading.Timer(1, fun_timer)
i=0
timer.start()

10.65.60.196 2 2020-04-03 17:34:45 superman (null) 1 1
10.65.60.196 3 2020-04-03 17:34:35 2 0.250000 40.423332 30.068600 1 9288 0 1
10.65.60.196 3 2020-04-03 17:34:34 1247 2.620968 45.689209 34.113781 1 14864 0 0
10.65.60.196 3 2020-04-03 17:34:25 2 0.277218 40.423332 30.069426 1 17656 0 1
10.65.60.196 3 2020-04-03 17:34:24 1247 3.224206 45.689209 34.119354 1 18544 0 0
10.65.60.196 3 2020-04-03 17:34:15 2 0.300000 40.423332 30.069220 1 16792 0 1
10.65.60.196 2 2020-04-03 17:34:17 superman (null) 1 3
10.65.60.196 3 2020-04-03 17:34:14 1247 3.050000 45.689209 34.132351 1 17272 0 0
10.65.60.196 3 2020-04-03 17:34:05 3 0.375000 40.423332 30.069838 1 4248 0 1


In [7]:
#方法2：
vup=VarUpdater(myvar)
i=0
vup.start()

In [ ]:
server.stop()